In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from time import time
from sklearn.ensemble import ExtraTreesRegressor
import pandas as pd
from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
# https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py

In [3]:
df = pd.read_csv("dataset_N2N.csv", sep=",", header=0)
df.head()

,x_s,time_s,Temp,X1,X2,X3,X4,X5,X6,X7,...,RVV38,RVV39,RVV40,RVV41,RVV42,RVV43,RVV44,RVV45,RVV46,RVV47
0,0.000000,0.000000e+00,10757.166,0.999986,0.000014,2.397178e-10,4.558197e-15,9.939829e-20,2.485752e-24,7.129016e-29,...,-2.401320e-109,-5.311691e-112,-1.346084e-114,-3.908330e-117,-1.300201e-119,-4.956191e-122,-2.164807e-124,-1.083521e-126,-6.214598e-129,-4.115686e-131
1,0.000004,8.216688e-12,10757.146,0.999963,0.000037,1.815259e-09,1.240808e-13,1.154856e-17,1.419631e-21,2.233567e-25,...,4.313327e+01,4.319762e+01,4.302709e+01,4.261090e+01,4.193997e+01,4.100697e+01,3.980402e+01,3.819891e+01,3.036801e+01,-1.093747e+03
2,0.000008,1.643338e-11,10757.126,0.999941,0.000059,4.418282e-09,4.266114e-13,5.241519e-17,8.059130e-21,1.524676e-24,...,2.222941e+02,2.225802e+02,2.216414e+02,2.194194e+02,2.158637e+02,2.109328e+02,2.045732e+02,1.958232e+02,1.487166e+02,-5.628267e+03
3,0.000012,2.465006e-11,10757.106,0.999919,0.000081,8.070145e-09,9.922813e-13,1.496790e-16,2.742066e-20,6.035964e-24,...,6.379328e+02,6.386028e+02,6.357109e+02,6.290790e+02,6.185514e+02,6.039950e+02,5.852063e+02,5.583745e+02,4.014412e+02,-1.612388e+04
4,0.000029,6.058392e-11,10757.020,0.999822,0.000178,3.553516e-08,8.802196e-12,2.765142e-15,1.098811e-18,5.460371e-22,...,8.370745e+03,8.365424e+03,8.308980e+03,8.198153e+03,8.029840e+03,7.800562e+03,7.495944e+03,6.918945e+03,3.138381e+03,-2.091199e+05


In [4]:
X = df.iloc[:,0:54].values    # x_s, time_s, Temp, ni_n, na_n, rho, v, p, E, H
y = df.iloc[:,54:55].values   # rhs[0:50]

In [5]:

# df2=pd.DataFrame(X)
# df2


In [6]:
feature_names = ["x_s","time_s", "Temp"] + [f"X{i}" for i in range(1, 48)] + ["rho", "v","p"] + [f"RDm{i}" for i in range(1, 47)] + [f"RDa{i}" for i in range(1, 47)] + [f"RVTm{i}" for i in range(1, 47)] + [f"RVTa{i}" for i in range(1, 47)] + [f"RVV{i}" for i in range(1, 47)]
list1 = list(feature_names)

In [7]:
est = ExtraTreesRegressor(n_estimators=50)
est = est.fit(X, y.ravel())

In [8]:
tic_fwd = time()
sfs_forward = SequentialFeatureSelector(est, n_features_to_select=2, direction="forward").fit(X, y.ravel())
toc_fwd = time()

In [9]:
tic_bwd = time()
sfs_backward = SequentialFeatureSelector(est, n_features_to_select=2, direction="backward").fit(X, y.ravel())
toc_bwd = time()

In [15]:
feature_names_v = list1[0:54:1]
a = np.array(feature_names_v)

In [16]:
sfs_forward.get_support()

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False])

In [17]:
sfs_backward.get_support()

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False])

In [18]:
# sfs_forward.get_params()

In [19]:
print(
    "Features selected by forward sequential selection: "
    f"{a[sfs_forward.get_support()]}"
)
print(f"Done in {toc_fwd - tic_fwd:.3f}s") 

Features selected by forward sequential selection: ['X7' 'rho']
Done in 30.940s


In [20]:
print(
    "Features selected by backward sequential selection: "
    f"{a[sfs_backward.get_support()]}"
)
print(f"Done in {toc_bwd - tic_bwd:.3f}s")


Features selected by backward sequential selection: ['X7' 'rho']
Done in 1681.586s


In [ ]:
# Сравнение SelectFromModel и SFS
# Чтобы закончить, мы должны отметить, что SelectFromModelэто значительно быстрее, чем SFS. Действительно, SelectFromModelнужно только соответствовать модели один раз, в то время как SFS должна перекрестно проверять множество различных моделей для каждой из итераций. Однако SFS работает с любой моделью, в то время SelectFromModelкак требует, чтобы базовый оценщик предоставлял coef_атрибут или feature_importances_атрибут. Прямой SFS быстрее, чем обратный SFS, потому что ему нужно только выполнять n_features_to_select = 2итерации, в то время как обратный SFS должен выполнять n_features - n_features_to_select = 8итерации.